In [ ]:
import mysql.connector
from mysql.connector import Error

# Function to connect to the database
def connect():
    try:
        connection = mysql.connector.connect(
            host='localhost',
            database='task_management',
            user='root',  # Replace with your MySQL username
            password='12345'  # Replace with your MySQL password
        )
        if connection.is_connected():
            print("Connected to the database")
            return connection
    except Error as e:
        print(f"Error: {e}")
        return None

# Function to add a new task
def add_task(title, description):
    connection = connect()
    if connection:
        cursor = connection.cursor()
        query = "INSERT INTO tasks (title, description) VALUES (%s, %s)"
        cursor.execute(query, (title, description))
        connection.commit()
        print(f"Task '{title}' added successfully!")
        cursor.close()
        connection.close()

# Function to view all tasks
def view_tasks():
    connection = connect()
    if connection:
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM tasks")
        tasks = cursor.fetchall()
        for task in tasks:
            print(f"ID: {task[0]}, Title: {task[1]}, Status: {task[3]}, Created: {task[4]}")
        cursor.close()
        connection.close()

# Function to update task status
def update_task_status(task_id, status):
    connection = connect()
    if connection:
        cursor = connection.cursor()
        query = "UPDATE tasks SET status = %s WHERE task_id = %s"
        cursor.execute(query, (status, task_id))
        connection.commit()
        print(f"Task ID {task_id} status updated to '{status}'")
        cursor.close()
        connection.close()

# Main function to handle the user input
def main():
    while True:
        print("\nTask Management System")
        print("1. Add a new task")
        print("2. View all tasks")
        print("3. Update task status")
        print("4. Exit")
        choice = input("Choose an option: ")

        if choice == '1':
            title = input("Enter task title: ")
            description = input("Enter task description: ")
            add_task(title, description)
        elif choice == '2':
            view_tasks()
        elif choice == '3':
            task_id = int(input("Enter task ID to update: "))
            status = input("Enter new status (Not Started, In Progress, Completed): ")
            update_task_status(task_id, status)
        elif choice == '4':
            break
        else:
            print("Invalid option, please try again.")

if __name__ == "__main__":
    main()



Task Management System
1. Add a new task
2. View all tasks
3. Update task status
4. Exit


In [1]:
import tkinter as tk
from tkinter import messagebox
import datetime

class Task:
    def __init__(self, task_id, title, description):
        self.task_id = task_id
        self.title = title
        self.description = description
        self.status = "Not Started"
        self.deadline = None
        self.reminder = None

    def __str__(self):
        return f"ID: {self.task_id}, Title: {self.title}, Description: {self.description}, Status: {self.status}"

class TaskManager:
    def __init__(self):
        self.tasks = []
        self.task_counter = 1

    def add_task(self, title, description):
        task = Task(self.task_counter, title, description)
        self.tasks.append(task)
        self.task_counter += 1

    def view_tasks(self):
        return [str(task) for task in self.tasks]

    def update_task_status(self, task_id, status):
        task = self.find_task_by_id(task_id)
        if task:
            task.status = status
            return f"Task ID {task_id} status updated to {status}"
        else:
            return f"Task ID {task_id} not found."

    def delete_task(self, task_id):
        task = self.find_task_by_id(task_id)
        if task:
            self.tasks.remove(task)
            return f"Task ID {task_id} deleted."
        else:
            return f"Task ID {task_id} not found."

    def find_task_by_id(self, task_id):
        for task in self.tasks:
            if task.task_id == task_id:
                return task
        return None

    def set_task_deadline(self, task_id, deadline):
        task = self.find_task_by_id(task_id)
        if task:
            task.deadline = deadline
            return f"Task ID {task_id} deadline set to {deadline}"
        else:
            return f"Task ID {task_id} not found."

    def set_task_reminder(self, task_id, reminder_time):
        task = self.find_task_by_id(task_id)
        if task:
            task.reminder = reminder_time
            return f"Task ID {task_id} reminder set to {reminder_time}"
        else:
            return f"Task ID {task_id} not found."

class TaskManagerGUI(tk.Tk):
    def __init__(self, task_manager):
        super().__init__()
        self.task_manager = task_manager
        self.title("Task Manager")
        self.geometry("500x500")

        # Title Label
        self.title_label = tk.Label(self, text="Task Manager", font=("Arial", 16))
        self.title_label.pack(pady=10)

        # Task listbox
        self.task_listbox = tk.Listbox(self, width=50, height=15)
        self.task_listbox.pack(pady=10)

        # Buttons
        self.add_button = tk.Button(self, text="Add Task", command=self.add_task)
        self.add_button.pack(pady=5)

        self.view_button = tk.Button(self, text="View All Tasks", command=self.view_tasks)
        self.view_button.pack(pady=5)

        self.update_button = tk.Button(self, text="Update Task Status", command=self.update_task_status)
        self.update_button.pack(pady=5)

        self.delete_button = tk.Button(self, text="Delete Task", command=self.delete_task)
        self.delete_button.pack(pady=5)

        self.exit_button = tk.Button(self, text="Exit", command=self.quit)
        self.exit_button.pack(pady=20)

    def add_task(self):
        def save_task():
            title = title_entry.get()
            description = description_entry.get()
            if title and description:
                self.task_manager.add_task(title, description)
                messagebox.showinfo("Task Added", "Task has been added successfully!")
                add_task_window.destroy()
                self.view_tasks()
            else:
                messagebox.showerror("Error", "Please provide both title and description.")
        
        # Create the Add Task window
        add_task_window = tk.Toplevel(self)
        add_task_window.title("Add Task")

        title_label = tk.Label(add_task_window, text="Title:")
        title_label.pack(pady=5)
        title_entry = tk.Entry(add_task_window)
        title_entry.pack(pady=5)

        description_label = tk.Label(add_task_window, text="Description:")
        description_label.pack(pady=5)
        description_entry = tk.Entry(add_task_window)
        description_entry.pack(pady=5)

        save_button = tk.Button(add_task_window, text="Save", command=save_task)
        save_button.pack(pady=10)

    def view_tasks(self):
        self.task_listbox.delete(0, tk.END)
        tasks = self.task_manager.view_tasks()
        for task in tasks:
            self.task_listbox.insert(tk.END, task)

    def update_task_status(self):
        def save_status():
            task_id = int(task_id_entry.get())
            status = status_entry.get()
            if status:
                message = self.task_manager.update_task_status(task_id, status)
                messagebox.showinfo("Status Updated", message)
                update_task_window.destroy()
                self.view_tasks()
            else:
                messagebox.showerror("Error", "Please enter a status.")

        # Create the Update Task Status window
        update_task_window = tk.Toplevel(self)
        update_task_window.title("Update Task Status")

        task_id_label = tk.Label(update_task_window, text="Task ID:")
        task_id_label.pack(pady=5)
        task_id_entry = tk.Entry(update_task_window)
        task_id_entry.pack(pady=5)

        status_label = tk.Label(update_task_window, text="New Status:")
        status_label.pack(pady=5)
        status_entry = tk.Entry(update_task_window)
        status_entry.pack(pady=5)

        save_button = tk.Button(update_task_window, text="Save", command=save_status)
        save_button.pack(pady=10)

    def delete_task(self):
        def delete_task_from_list():
            task_id = int(task_id_entry.get())
            message = self.task_manager.delete_task(task_id)
            messagebox.showinfo("Task Deleted", message)
            delete_task_window.destroy()
            self.view_tasks()

        # Create the Delete Task window
        delete_task_window = tk.Toplevel(self)
        delete_task_window.title("Delete Task")

        task_id_label = tk.Label(delete_task_window, text="Task ID:")
        task_id_label.pack(pady=5)
        task_id_entry = tk.Entry(delete_task_window)
        task_id_entry.pack(pady=5)

        delete_button = tk.Button(delete_task_window, text="Delete", command=delete_task_from_list)
        delete_button.pack(pady=10)

if __name__ == "__main__":
    task_manager = TaskManager()
    app = TaskManagerGUI(task_manager)
    app.mainloop()
